# Import Libraries

In [7]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Function

In [8]:
# Function for querying
client = bigquery.Client()

def get_data(query):
    """
    Retrieve data from a query using a specified client and return as a DataFrame.

    Parameters:
        query (str): The query to execute.

    Returns:
        pandas.DataFrame: DataFrame containing the retrieved data.
    """
    return client.query(query).to_dataframe()

# Function to retrive year only in date
def get_year(df, drop=True):
    """
    Extract the year from the 'date' column in the DataFrame and create a new 'year' column.

    Parameters:
        df (pandas.DataFrame): Input DataFrame containing a 'date' column.
        drop (bool): Whether to drop the original 'date' column. Default is True.

    Returns:
        pandas.DataFrame: DataFrame with a new 'year' column and the option to drop the 'date' column.
    """
    df['year'] = pd.DatetimeIndex(df['date']).year.astype(int)
    if drop:
        df.drop(columns='date', inplace=True)
    return df

def get_unique_diff(data, unique_dates, unique_provinsi, unique_kota):
    """
    Calculate the difference in unique values between the input data and specified unique values.

    Parameters:
        data (pandas.DataFrame): Input DataFrame.
        unique_dates (list): List of unique dates.
        unique_provinsi (list): List of unique 'provinsi' values.
        unique_kota (list): List of unique 'kota' values.

    Returns:
        None (prints differences between input data and unique values).
    """
    year_diff = set(data['year'].unique()) ^ set(unique_dates)
    provinsi_diff = set(data['provinsi'].unique()) ^ set(unique_provinsi)
    kota_diff = set(data['kota'].unique()) ^ set(unique_kota)

    print(f"Unique values in 'year' column (Difference): {year_diff}")
    print(f"Unique values in 'provinsi' column (Difference): {provinsi_diff}")
    print(f"Unique values in 'kota' column (Difference): {kota_diff}")


def check_dataframe(df):
    """
    Check DataFrame for duplicates, NaN, None, or "" values in each column.
    
    Parameters:
        df (pandas.DataFrame): Input DataFrame
        
    Returns:
        str: Human-readable summary of DataFrame check results.
    """
    output = ""
    
    has_duplicates = df.duplicated().any()
    
    if has_duplicates:
        output += "DataFrame contains duplicates\n"
    else:
        output += "No duplicates found in DataFrame\n"
    
    for column in df.columns:
        has_nan = df[column].isnull().any()
        has_none = df[column].apply(lambda x: x is None).any()
        has_empty_string = (df[column] == "").any()
        
        column_result = f"Column '{column}':"
        
        if has_nan or has_none or has_empty_string:
            column_result += "\n"
            if has_nan:
                column_result += "    - Contains NaN values\n"
            if has_none:
                column_result += "    - Contains None values\n"
            if has_empty_string:
                column_result += "    - Contains empty string values\n"
        else:
            column_result += " No issues found\n"
        
        output += column_result
    
    return output

# Get Data

## Get Base Data

In [9]:
## Query Data -------------------------------------------
# Get data
katadata_food = get_data("SELECT provinsi, kota, date, value FROM `katadata-sales-planning.sandbox_spt.raw_rerata_pengeluaran_perkapita_sebulan_makanan_kabkota`")
katadata_non_food = get_data("SELECT provinsi, kota, date, value FROM `katadata-sales-planning.sandbox_spt.raw_rerata_pengeluaran_perkapita_sebulan_non_makanan_kabkota`")
katadata_total = get_data("SELECT provinsi, kota, date, value FROM `katadata-sales-planning.sandbox_spt.raw_rerata_pengeluaran_perkapita_sebulan_total_kabkota`")
katadata_prepared_food = get_data("SELECT provinsi, kota, date, value FROM `katadata-sales-planning.sandbox_spt.raw_rerata_pengeluaran_perkapita_sebulan_makanan_jadi_kabkota`")

## Adjust provinsi -------------------------------------------
# Create a dictionary mapping 'kota' to 'provinsi' in katadata_food
kota_to_provinsi_mapping = dict(zip(katadata_food['kota'], katadata_food['provinsi']))

# Update 'provinsi' column in katadata_prepared_food based on the mapping
katadata_prepared_food['provinsi'] = katadata_prepared_food['kota'].map(kota_to_provinsi_mapping)

## Adjust date -------------------------------------------
# Get year from date
for data in [katadata_prepared_food, katadata_food, katadata_non_food, katadata_total]:
    data = get_year(data)

# Filter `katadata_prepared_food` to cover the period from 2019 to 2023
katadata_prepared_food = katadata_prepared_food[(katadata_prepared_food['year'] >= 2018) & (katadata_prepared_food['year'] <= 2023)]

# Filter predictor dataframes to cover the period from 2018 to 2023
katadata_food = katadata_food[(katadata_food['year'] >= 2018) & (katadata_food['year'] <= 2023)]
katadata_non_food = katadata_non_food[(katadata_non_food['year'] >= 2018) & (katadata_non_food['year'] <= 2023)]
katadata_total = katadata_total[(katadata_total['year'] >= 2018) & (katadata_total['year'] <= 2023)]

## Merge dataframes -------------------------------------------
# Preprocess all values in the DataFrames
for data in [katadata_prepared_food, katadata_food, katadata_non_food, katadata_total]:
    data['value'] = data['value'].astype(float)
    data['value'] = (data['value'] / 30) * 7 * 52

# Rename columns for clarity and merge DataFrames
merged_data = (
    katadata_prepared_food.rename(columns={'value': 'prepared_food_exp'})
    .merge(
        katadata_food.rename(columns={'value': 'food_exp'}),
        on=['provinsi', 'kota', 'year'], how='left'
    )
    .merge(
        katadata_non_food.rename(columns={'value': 'non_food_exp'}),
        on=['provinsi', 'kota', 'year'], how='left'
    )
    .merge(
        katadata_total.rename(columns={'value': 'total_exp'}),
        on=['provinsi', 'kota', 'year'], how='left'
    )
)

# Adjust column order and sort the DataFrame
base_data = (
    merged_data[['year', 'provinsi', 'kota', 'prepared_food_exp',
                 'food_exp', 'non_food_exp', 'total_exp']]
    .groupby(['provinsi', 'kota', 'year']).max().reset_index()
)

In [10]:
# Get unique values of data
base_data_year_unique = base_data['year'].unique()
base_data_provinsi_unique = base_data['provinsi'].unique()
base_data_kota_unique = base_data['kota'].unique()

## Get Podes & Susenas Data

In [64]:
## Time frame per year
# Get Podes data
jumlah_restaurant = get_data("SELECT year, kota_kab, prov, restaurant FROM `katadata-sales-planning.sandbox_spt.raw_data_podes`")
jumlah_intansi_pendidikan = get_data("SELECT year, kota_kab, prov, elementary, high_school, mid_school, perguruan_tinggi FROM `katadata-sales-planning.sandbox_spt.raw_data_podes`")

# Get Susenas data
pdrb = get_data("SELECT nama, provinsi, kota, date, value FROM `katadata-sales-planning.sandbox_spt.raw_pdrb_adhk_kabkota`")
jumlah_penduduk = get_data("SELECT provinsi, kota, date, value FROM `katadata-sales-planning.sandbox_spt.raw_jumlah_penduduk_kabkota`")
penduduk_laki_laki = get_data("SELECT provinsi, date, value FROM `katadata-sales-planning.sandbox_spt.raw_jumlah_penduduk_laki_laki_provinsi`")
penduduk_perempuan = get_data("SELECT provinsi, date, value FROM `katadata-sales-planning.sandbox_spt.raw_jumlah_penduduk_perempuan_provinsi`")
kategori_usia = get_data("SELECT nama, provinsi, kota, date, value FROM `katadata-sales-planning.sandbox_spt.raw_jumlah_penduduk_kategori_usia_kabkota`")

## Timeframe per month
# Get Susenas data
indeks_harga_konsumen = get_data("SELECT kota, provinsi, date, value FROM `katadata-sales-planning.sandbox_spt.raw_indeks_harga_konsumen_kabkota`")
inflasi = get_data("SELECT date, provinsi, kota, value FROM `katadata-sales-planning.katadata_sales_planning.p2_inflasi_ytd_90kota`")

## Initial preprocessing
# date to year
for data in [pdrb, jumlah_penduduk, penduduk_laki_laki, penduduk_perempuan, kategori_usia, indeks_harga_konsumen, inflasi]:
    try:
        data = get_year(data)
    except:
        pass

# Preprocess all values in the DataFrames
for data in [pdrb, indeks_harga_konsumen, inflasi]:
    try:
        data['value'] = data['value'].astype(float)
    except:
        pass
    
for data in [jumlah_restaurant, jumlah_intansi_pendidikan, jumlah_penduduk, penduduk_laki_laki, penduduk_perempuan, kategori_usia]:
    try:
        data['value'] = data['value'].astype(float)
        data['value'] = data['value'].astype(int)
    except:
        pass

# Data Preprocessing

## PDRB

In [18]:
# Basic check
print(check_dataframe(pdrb))

No duplicates found in DataFrame
Column 'nama': No issues found
Column 'provinsi': No issues found
Column 'kota': No issues found
Column 'date':
    - Contains empty string values
Column 'value':
    - Contains empty string values



pdrb_fnbThe PDRB sectors that will be used are **PDRB ADHK Sektor Industri Pengolahan Menurut Kabupaten Kota** with the related subsector being **Industri Makanan dan Minuman**, and **PDRB ADHK Sektor Penyediaan Akomodasi dan Makan Minum Menurut Kabupaten Kota** with the related subsector being **Penyediaan Makan Minum**, as they are related to spending on food and beverages.

In [19]:
# Handle issues
pdrb = pdrb.replace({'None': 0, np.nan: 0, '': 0})
pdrb = get_year(pdrb)

# Filter and make nama as a column
pdrb_fnb = pdrb[(pdrb['nama'] == 'PDRB ADHK Sektor Penyediaan Akomodasi dan Makan Minum Menurut Kabupaten Kota') | (pdrb['nama'] == 'PDRB ADHK Sektor Industri Pengolahan Menurut Kabupaten Kota')]
pdrb_fnb = pdrb_fnb.pivot_table(index=['provinsi', 'kota', 'year'], columns='nama', values='value', aggfunc='first').reset_index().rename_axis(None, axis=1)
pdrb_fnb = pdrb_fnb.rename(columns = {'PDRB ADHK Sektor Industri Pengolahan Menurut Kabupaten Kota':'pdrb_adhk_pengolahan', 'PDRB ADHK Sektor Penyediaan Akomodasi dan Makan Minum Menurut Kabupaten Kota':'pdrb_adhk_penyediaan'})

# Change values type from object to float
for col in pdrb_fnb.iloc[:,3:].columns:
    pdrb_fnb[col] = pdrb_fnb[col].astype(float)

# Filter timeframe
pdrb_fnb['year'] = pdrb_fnb['year'].astype(int)
pdrb_fnb = pdrb_fnb[pdrb_fnb['year']>=2018]

# Combine pdrb_adhk_pengolahan & pdrb_adhk_penyediaan as pdrb_adhk_prepare_food
pdrb_fnb['pdrb_adhk_prepare_food'] = pdrb_fnb['pdrb_adhk_pengolahan'] + pdrb_fnb['pdrb_adhk_penyediaan']

Final checking of `pdrb_fnb` differences with target data will be based on the unique values of year, provinsi, and kota.

In [20]:
get_unique_diff(pdrb_fnb, base_data_year_unique, base_data_provinsi_unique, base_data_kota_unique)

Unique values in 'year' column (Difference): {2023}
Unique values in 'provinsi' column (Difference): set()
Unique values in 'kota' column (Difference): set()


The 2023 data will be filled by extrapolation in the later step.

## Jumlah Penduduk

In [21]:
# Basic check
print(check_dataframe(jumlah_penduduk))

DataFrame contains duplicates
Column 'provinsi': No issues found
Column 'kota': No issues found
Column 'value': No issues found
Column 'year': No issues found



In [65]:
# Handle issues
jumlah_penduduk = jumlah_penduduk[~jumlah_penduduk.duplicated()]
jumlah_penduduk.rename(columns = {'value':'jumlah_penduduk'}, inplace = True)

# Filter timeframe
jumlah_penduduk = jumlah_penduduk[jumlah_penduduk['year']>=2018]

Final checking of `jumlah_penduduk` differences with target data will be based on the unique values of year, provinsi, and kota.

In [23]:
get_unique_diff(jumlah_penduduk, base_data_year_unique, base_data_provinsi_unique, base_data_kota_unique)

Unique values in 'year' column (Difference): {2022}
Unique values in 'provinsi' column (Difference): {'Kalimantan Utara'}
Unique values in 'kota' column (Difference): set()


The 2022 data will be filled by extrapolation in the later step.

## Jumlah Penduduk Laki-Laki & Perempuan

In [24]:
# Basic check
print(check_dataframe(penduduk_laki_laki))

No duplicates found in DataFrame
Column 'provinsi': No issues found
Column 'value': No issues found
Column 'year': No issues found



In [25]:
# Basic check
print(check_dataframe(penduduk_perempuan))

No duplicates found in DataFrame
Column 'provinsi': No issues found
Column 'value': No issues found
Column 'year': No issues found



In [66]:
jumlah_penduduk

,provinsi,kota,jumlah_penduduk,year
1,Jawa Barat,Kab. Pangandaran,401493,2020
2,Jawa Barat,Kab. Pangandaran,399284,2019
3,Jawa Barat,Kab. Pangandaran,397187,2018
12,Jawa Barat,Kab. Pangandaran,433091,2021
13,Jawa Barat,Kab. Pangandaran,440177,2023
...,...,...,...,...
15332,Kep. Bangka Belitung,Kab. Bangka Barat,209011,2018
15333,Kep. Bangka Belitung,Kab. Bangka Barat,213163,2019
15334,Kep. Bangka Belitung,Kab. Bangka Barat,217332,2020
15335,Kep. Bangka Belitung,Kab. Bangka Barat,206937,2021


The data needs to be handled further to create city-level referencing based on jumlah penduduk

In [68]:
jumlah_penduduk_provinsi = get_data("SELECT provinsi, date, value FROM `katadata-sales-planning.sandbox_spt.raw_jumlah_penduduk_provinsi`")

In [82]:
# Create jumlah penduduk at province level
jumlah_penduduk_provinsi_groupby_manual = jumlah_penduduk.groupby(['provinsi', 'year'])[['jumlah_penduduk']].sum().reset_index()
jumlah_penduduk_provinsi_groupby_manual.rename(columns = {'jumlah_penduduk':'jumlah_penduduk_groupby'}, inplace = True)

In [81]:
jumlah_penduduk_provinsi = get_year(jumlah_penduduk_provinsi)

KeyError: 'date'

In [83]:
jumlah_penduduk_provinsi.rename(columns = {'value':'jumlah_penduduk_query'}, inplace = True)

In [90]:
jumlah_penduduk_provinsi['jumlah_penduduk_query'] = jumlah_penduduk_provinsi['jumlah_penduduk_query'].astype(float)

In [124]:
jumlah_penduduk_provinsi = jumlah_penduduk_provinsi[~jumlah_penduduk_provinsi.duplicated(subset=['provinsi', 'year'])]

In [125]:
comparison_jumlah = jumlah_penduduk_provinsi_groupby_manual.merge(jumlah_penduduk_provinsi, on=['provinsi', 'year'], how='left')

In [126]:
comparison_jumlah['diff'] = (abs(comparison_jumlah['jumlah_penduduk_groupby'] - comparison_jumlah['jumlah_penduduk_query']) / comparison_jumlah['jumlah_penduduk_query']) * 100

In [127]:
comparison_jumlah = comparison_jumlah[comparison_jumlah['year']>=2018]

In [128]:
comparison_jumlah = comparison_jumlah.dropna()

In [129]:
comparison_jumlah[comparison_jumlah.isna().any(axis=1)]

,provinsi,year,jumlah_penduduk_groupby,jumlah_penduduk_query,diff


In [130]:
comparison_jumlah.describe()

,year,jumlah_penduduk_groupby,jumlah_penduduk_query,diff
count,119.00,119.00,119.00,119.00
mean,2019.45,8442526.71,8353780.88,4.55
std,1.17,11783134.15,11681041.66,10.47
min,2018.00,30896.00,941392.00,0.00
25%,2018.00,2363295.00,2426588.00,0.51
50%,2019.00,4336900.00,4162359.00,1.63
75%,2021.00,8419613.00,8477377.00,3.92
max,2021.00,49935858.00,50103066.00,97.76


In [131]:
comparison_jumlah.sort_values(by=['diff'], ascending=False)

,provinsi,year,jumlah_penduduk_groupby,jumlah_penduduk_query,diff
117,Sulawesi Barat,2020,30896,1378107.00,97.76
104,Papua,2021,4577243,3438241.00,33.13
64,Kalimantan Timur,2020,4530761,3664669.00,23.63
65,Kalimantan Timur,2021,4574427,3708932.00,23.34
63,Kalimantan Timur,2019,4434708,3619676.00,22.52
...,...,...,...,...,...
43,Jawa Timur,2018,39500851,39521647.00,0.05
31,Jambi,2021,3642763,3641271.00,0.04
144,Sumatera Selatan,2019,8496898,8497173.00,0.00
25,Gorontalo,2018,1166142,1166139.00,0.00


In [132]:
comparison_jumlah['diff'].median()

1.6277980054335468

In [ ]:
# Handle penduduk laki-laki
penduduk_laki_laki = penduduk_laki_laki.rename(columns = {'value':'penduduk_laki_laki_provinsi'})
penduduk_laki_laki = penduduk_laki_laki.merge(jumlah_penduduk_provinsi, on = ['year', 'provinsi'], how = 'inner').merge(jumlah_penduduk, on = ['year', 'provinsi'], how = 'inner')
penduduk_laki_laki['penduduk_laki_laki'] = penduduk_laki_laki['jumlah_penduduk'] / penduduk_laki_laki['jumlah_penduduk_provinsi']*penduduk_laki_laki['penduduk_laki_laki_provinsi']
penduduk_laki_laki.drop(columns = ['penduduk_laki_laki_provinsi', 'jumlah_penduduk_provinsi', 'jumlah_penduduk'], inplace = True)

# Handle penduduk perempuan
penduduk_perempuan['year'] = pd.DatetimeIndex(penduduk_perempuan['date']).year
penduduk_perempuan.drop(columns = 'date', inplace = True)
penduduk_perempuan.rename(columns = {'value':'penduduk_perempuan_provinsi'}, inplace = True)
penduduk_perempuan['penduduk_perempuan_provinsi'] = penduduk_perempuan['penduduk_perempuan_provinsi'].astype(float)
penduduk_perempuan = penduduk_perempuan.merge(jumlah_penduduk_provinsi, on = ['year', 'provinsi'], how = 'inner').merge(jumlah_penduduk, on = ['year', 'provinsi'], how = 'inner')
penduduk_perempuan['penduduk_perempuan'] = penduduk_perempuan['jumlah_penduduk']/penduduk_perempuan['jumlah_penduduk_provinsi']*penduduk_perempuan['penduduk_perempuan_provinsi']
penduduk_perempuan.drop(columns = ['penduduk_perempuan_provinsi', 'jumlah_penduduk_provinsi', 'jumlah_penduduk'], inplace = True)